In [56]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from collections import Counter

import functools

import math

In [52]:
inputFile = "ID_data_mass_18122012_prepared.csv"

dataFrame = pd.read_csv(inputFile, header = 0, sep = ';')
print(dataFrame.shape)
data = dataFrame.values[0:10,0:3]
print(data)

numSamples = dataFrame.shape[0]
print("Num samples == " + str(numSamples))
numFeatures = dataFrame.shape[1]
print("Num features == " + str(numFeatures))

(185, 33)
[[804 '05.06.08' '3576.30']
 [804 '06.06.08' '3576.30']
 [804 '07.06.08' '3576.30']
 [804 '08.06.08' '3576.30']
 [804 '09.06.08' '3576.30']
 [804 '17.06.11' '3576.30']
 [807 '16.09.04' '3590.00']
 [807 '16.09.04' '3590.00']
 [807 '17.09.04' '3590.00']
 [807 '17.09.04' '3590.00']]
Num samples == 185
Num features == 33


In [59]:
kgfColumnId = dataFrame.shape[1] - 1
gtotalColumnId = dataFrame.shape[1] - 2

listOfKeys = [str(line[kgfColumnId]) + '/' + str(line[gtotalColumnId]) for line in dataFrame.values]
uniqueKeys = Counter(listOfKeys).keys()

keyFrequencies = Counter(listOfKeys).values()
print(keyFrequencies)
print(uniqueKeys)

probabilities = [keyFrequency / numSamples for keyFrequency in keyFrequencies]
print(probabilities)

entropy = functools.reduce(lambda accum, element : accum + (-element * math.log2(element)), probabilities)
print("Entropy == " + str(entropy))

dict_values([1, 1, 1, 1, 1, 1, 92, 1, 2, 2, 2, 1, 3, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 4, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1])
dict_keys(['311.91/2.78', '288.6/3.7', '248.79/4.52', '223.56/5.22', '215.15/5.77', '241.13/3.08', 'nan/nan', '188.0/nan', '178.0/nan', '169.0/nan', '162.0/nan', '230.0/nan', '201.0/nan', '172.0/nan', '157.0/nan', '141.0/nan', '385.42/4.23', '295.5/4.71', '255.68/5.73', '233.96/5.78', '223.11/5.76', '199.63/5.25', '245.68/4.25', '294.59/3.72', '282.24/5.17', '280.56/8.4', '292.24/9.64', '284.08/10.93', '289.18/9.33', '287.52/6.42', '254.84/4.59', '243.23/5.83', '219.04/7.28', '270.0/nan', '251.0/nan', '236.0/nan', '217.0/nan', '200.0/nan', '180.0/nan', '160.0/nan', '156.0/nan', '153.0/nan', '152.0/nan', '138.0/nan', '131.0/nan', '212.0/nan', '194.0/nan', '187.0/nan', '198.0/nan', '174.0/nan', '171.0/nan', '165.0/nan', '140.0/nan', '154.0/nan', '139.0/nan',